In [4]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
from numpy.linalg import norm
from vis_lorenz_LRDelta import read_csv

In [5]:
def flip_vecs(vecs):
    for i in range(1, vecs.shape[0]):
        for j in range(vecs.shape[1]):
            flip = norm(vecs[i, j] + vecs[i-1, j]
                        ) < norm(vecs[i, j] - vecs[i-1, j])
            if flip:
                vecs[i, j] *= -1
    return vecs

In [6]:
us = read_csv("drive-ks-seq.out")
vs = read_csv("drive-ks-lv-seq.out")
u = read_csv("drive-ks.out")
v = read_csv("drive-ks-lv.out")

nt, nx = us.shape
rank = v.shape[1] // nx
# v = flip_vecs(v.reshape(nt, rank, nx))
# vs = flip_vecs(vs.reshape(nt, rank, nx))
vs = vs.reshape(nt, rank, nx)
v = v.reshape(nt, rank, nx)

In [7]:
stride = 2
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(5,5))
ax1.imshow(u[::-stride], cmap="plasma", aspect="auto")
ax2.imshow(us[::-stride], cmap="plasma", aspect="auto")
ax1.grid(False)
ax2.grid(False)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.figure()
plt.semilogy(np.linalg.norm(u - us, axis=1))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
extent = (0, 8, 0, 64)
x, t = np.linspace(0, 64, num=nx), np.linspace(0, 8, num=nt//stride)
X, T = np.meshgrid(x, t)

fig, axs = plt.subplots(3, 1)
(ax1, ax2, ax3) = axs
fig.tight_layout()
# ax1.imshow(u[::stride], cmap="viridis", extent=extent, aspect="auto")
# ax2.s(us[::stride], cmap="viridis", extent=extent, aspect="auto")

@widgets.interact(ind=(0, rank-1, 1))
def update(ind=0):
    [ax.cla() for ax in axs]
    ax1.imshow(vs[::stride, ind, :].T, cmap="plasma", aspect="auto", extent=extent)
    ax2.imshow(v[::stride, ind, :].T, cmap="plasma", aspect="auto", extent=extent)
    err = [norm(1 - np.abs(np.dot(v[i, ind], vs[i, ind].T))) for i in range(nt-1)]
    # err = [norm(v[i, ind] - vs[i, ind])/norm(vs[i, ind]) for i in range(nt-1)]
    ax3.semilogy(t, err[::stride])
    ax3.set_ylim((1e-16, 1e1))
    ax3.set_xlim((0, 8))
    ax1.grid(False)
    ax2.grid(False)
    # ax1.contour(X, T, vs[:-(1+nt%2):stride, ind])
    # ax2.contour(X, T, v[:-(1+nt%2):stride, ind])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='ind', max=15), Output()), _dom_classes=('widget-interact…

In [10]:
inds = range(12)
orth = [norm(np.eye(rank) - v[i, ...] @ v[i, ...].T) for i in range(nt-1)]
err = [norm(np.eye(inds[-1]+1) - np.dot(v[i, inds], vs[i, inds].T)) for i in range(nt-1)]

plt.figure()
plt.semilogy(err)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
def modarcos(arg):
    return np.arccos(np.pi*(arg % np.pi))
plt.figure()
errs = np.zeros(rank)
for j in range(0, rank, 8):
    plt.semilogy(np.abs([1 - np.dot(vs[i, j], v[i, j].T) for i in range(nt)]), label=f"{j}")
    # errs[j] = np.arccos(np.clip([np.dot(vs[i, j], v[i, j].T) for i in range(nt)], -1, 1)

# plt.semilogy(errs)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
j = 1
np.arccos(np.clip([np.dot(vs[i, j], v[i, j].T) for i in range(nt)], -1, 1))

array([2.107e-08, 0.000e+00, 2.107e-08, 0.000e+00, 0.000e+00, 2.107e-08,
       1.490e-08, 2.107e-08, 1.490e-08, 0.000e+00, 0.000e+00, 0.000e+00,
       1.490e-08, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.490e-08,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 2.107e-08, 1.490e-08,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 2.581e-08,
       0.000e+00, 2.107e-08, 0.000e+00, 0.000e+00, 2.581e-08, 2.107e-08,
       2.107e-08, 2.107e-08, 2.107e-08, 1.490e-08, 2.107e-08, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 2.107e-08, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.490e-08, 0.000e+00, 0.000e+00, 0.000e+00,
       1.490e-08, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.490e-08,
       2.107e-08, 0.000e+00, 0.000e+00, 2.107e-08, 0.000e+00])

In [13]:
n = 16
D = -2*np.eye(n) + np.diag(np.ones(n-1), -1) + np.diag(np.ones(n-1), 1)
D[0, -1] = 1
D[-1, 0] = 1
D

array([[-2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.],
       [ 1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0

In [14]:
W, V = np.linalg.eigh(D)
W = W[::-1]
V = V[:,::-1].T

In [15]:
W

array([ 1.665e-16, -1.522e-01, -1.522e-01, -5.858e-01, -5.858e-01,
       -1.235e+00, -1.235e+00, -2.000e+00, -2.000e+00, -2.765e+00,
       -2.765e+00, -3.414e+00, -3.414e+00, -3.848e+00, -3.848e+00,
       -4.000e+00])

In [16]:
eyes = [0, 1, 2, 3]
plt.figure()
for i in eyes:
    plt.plot(V[i]/np.max(V[i]), label=f"{i}")
plt.ylim([-1.1, 1.1])
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
V[:, 0]

array([-2.500e-01, -1.169e-02,  3.534e-01, -2.650e-04,  3.536e-01,
        3.536e-01, -1.055e-03,  3.535e-01, -2.809e-03, -1.308e-03,
        3.536e-01, -2.402e-01,  2.595e-01, -3.536e-01,  2.533e-04,
       -2.500e-01])